### Import Libraries

In [2]:
import pandas as pd
import numpy as np
import Saba_Scrapping as SS
from geopy.geocoders import Nominatim
import routing_osrm as Rosrm

### Scrap Data --> Infosaba.com

In [3]:
my_chromdriver = "Z:\\HQ\\BDM\\a.zohdi\\Data Engineering\\Github\\Geocoding\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe"

df_industries = SS.GET_industries(my_chromdriver)
print('Industries are Done !')

urls = df_industries['URL']
df_units = SS.GET_unit_subjects(urls, my_chromdriver)
print('Units are Done !')

df_complexes = SS.GET_complexes(my_chromdriver)
print('Complexes are Done !')

urls = df_units['URL']
df_unitcomplex = SS.GET_unit_complex(urls, my_chromdriver)
print('Unit_Complex is Done !')

Industries are Done !
Units are Done !
Complexes are Done !
There is no Complex in this link: https://infosaba.com/Unit_Subjects/بازیافت-پروپیلن
There is no Complex in this link: https://infosaba.com/Unit_Subjects/بوتانول
There is no Complex in this link: https://infosaba.com/Unit_Subjects/Staple
There is no Complex in this link: https://infosaba.com/Unit_Subjects/اپکسی-رزین
There is no Complex in this link: https://infosaba.com/Unit_Subjects/تاسیسات-استخراج-سنگ-آهن
There is no Complex in this link: https://infosaba.com/Unit_Subjects/ورق-خودرو
There is no Complex in this link: https://infosaba.com/Unit_Subjects/مشخصات-معادن-بوکسیت
There is no Complex in this link: https://infosaba.com/Unit_Subjects/کوره‌-شمش-ریزی-آلومینیوم
There is no Complex in this link: https://infosaba.com/Unit_Subjects/اکسیداسیون
There is no Complex in this link: https://infosaba.com/Unit_Subjects/بروشوئی-سیائیدی
There is no Complex in this link: https://infosaba.com/Unit_Subjects/خردایش
There is no Complex in thi

### Get Coordinations using complex.name & OSM.name (kinda)

In [2]:
complex_coords = pd.read_excel("Z:\\HQ\\BDM\\a.zohdi\\Complex OSM.xlsx", index_col=0)
geolocator = Nominatim(user_agent="Zohdi")

# getting coordinations of complexes
complex_coords['coordination'] = np.nan
i = 0
while i < len(complex_coords):
    if not pd.isna(complex_coords.iloc[i, 3]):

        OSM_name = complex_coords.iloc[i, 3]
        location = geolocator.geocode(OSM_name)
        coordination = f"{location.latitude}, {location.longitude}"
        complex_coords.iloc[i, 5] = coordination
        i += 1

    else:

        complex_coords.iloc[i, 5] = complex_coords.iloc[i, 4]
        i += 1

# split coordination to lat-long
complex_coords[['latitude', 'longitude']] = complex_coords['coordination'].str.split(',', expand=True)

In [9]:
df_complexes = complex_coords

with pd.ExcelWriter('Z:\\HQ\\BDM\\00 Public BDM\\A.Zohdi\\Saba Info.xlsx') as writer:
    df_industries.to_excel(writer, sheet_name='Industry')
    df_units.to_excel(writer, sheet_name='Unit')
    df_complexes.to_excel(writer, sheet_name='Complex')
    df_unitcomplex.to_excel(writer, sheet_name='Unit_Complex')

In [3]:
df_industries = pd.read_excel('Z:\\HQ\\BDM\\00 Public BDM\\A.Zohdi\\Saba Info.xlsx', sheet_name='Industry', index_col=0)
df_units = pd.read_excel('Z:\\HQ\\BDM\\00 Public BDM\\A.Zohdi\\Saba Info.xlsx', sheet_name='Unit', index_col=0)
df_complexes = pd.read_excel('Z:\\HQ\\BDM\\00 Public BDM\\A.Zohdi\\Saba Info.xlsx', sheet_name='Complex', index_col=0)
df_unitcomplex = pd.read_excel('Z:\\HQ\\BDM\\00 Public BDM\\A.Zohdi\\Saba Info.xlsx', sheet_name='Unit_Complex', index_col=0)

### Slicing Unit_Complex Dataframe for each Unit

In [4]:
# Transform capacity_value & under_construction_value 

df_unitcomplex['capacity_value'] = df_unitcomplex['capacity_value'].str.replace(",","")
df_unitcomplex['under_construction_value'] = df_unitcomplex['under_construction_value'].str.replace(",","")

# Transform coordination

df_complexes['coordination'] = df_complexes['coordination'].str.split(", ")

In [5]:
df_unitcomplex_joined = df_unitcomplex.merge(df_complexes, left_on='complex.name', right_on='name')
df_unitcomplex_joined.drop(columns=['unit.url', 'name', 'city', 'industry.name', 'OSM_name', 'GoogleMap_coordination'], inplace=True)

for i in df_units[df_units['industry.id'] == 4].index:
    exec("df_unitcomplex_"+str(i)+" = df_unitcomplex_joined[df_unitcomplex_joined['unit.id'] == i]")
    exec("df_unitcomplex_"+str(i)+".dropna(subset=['coordination'], inplace=True)")

<string>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<string>:1: SettingWithCopyW

### Test Run
complexes distance from unit=143 to unit=145

In [6]:
result = Rosrm.table(sources=list(df_unitcomplex_143['coordination']),
                     sources_reference=list(df_unitcomplex_143['complex.name']),
                     destinations=list(df_unitcomplex_145['coordination']),
                     destinations_reference=list(df_unitcomplex_145['complex.name']))

In [7]:
result['distance_km']

,مجتمع فولاد مبارکه,مجتمع هفت الماس,مجتمع گروه صنعتی پارلو,مجتمع صنعتی فولاد بهمن
مجتمع فولاد مبارکه,0.0000,551.0578,385.0453,772.2320
مجتمع فولاد گیلان,691.1635,188.8095,305.9870,368.1937
مجتمع هفت الماس,554.0146,0.0000,179.3799,369.6886
مجتمع فولاد امیرکبیر کاشان,259.7309,356.7991,214.5642,528.0291
مجتمع سازه صنعت مسیحا,497.7053,171.8072,116.9550,265.3071
مجتمع ورق خودرو چهارمحال و بختیاری,71.7260,576.4146,410.4022,797.5888
مجتمع فولاد دشتستان,426.9080,148.7090,44.9285,335.6844
مجتمع فولاد تاراز چهارمحال,85.3908,590.7064,424.6940,811.8806
مجتمع فولادی روی اندود کاوه,388.7466,181.5729,2.6908,366.4527
مجتمع صنایع فولاد شهریار تبریز,989.5861,527.1011,604.4096,890.1408


In [8]:
result['duration_h']

,مجتمع فولاد مبارکه,مجتمع هفت الماس,مجتمع گروه صنعتی پارلو,مجتمع صنعتی فولاد بهمن
مجتمع فولاد مبارکه,0.000000,6.169000,4.457278,9.005972
مجتمع فولاد گیلان,8.156500,2.350417,3.817389,6.180278
مجتمع هفت الماس,6.197167,0.000000,2.025444,5.408222
مجتمع فولاد امیرکبیر کاشان,3.071861,3.963778,2.713972,6.584750
مجتمع سازه صنعت مسیحا,5.486194,2.113667,1.586278,4.261000
مجتمع ورق خودرو چهارمحال و بختیاری,1.170889,6.681222,4.969500,9.518194
مجتمع فولاد دشتستان,5.057500,1.754250,0.723667,4.997611
مجتمع فولاد تاراز چهارمحال,1.271222,6.752167,5.040444,9.589139
مجتمع فولادی روی اندود کاوه,4.565167,2.137694,0.137333,5.336306
مجتمع صنایع فولاد شهریار تبریز,11.466389,5.971139,7.127278,11.232861


### Test Run
complexes routing from 'مجتمع فولاد مبارکه' to 'مجتمع هفت الماس'

In [9]:
origin = df_complexes.loc[df_complexes['name'] == 'مجتمع فولاد مبارکه', 'coordination'].values[0]
destination = df_complexes.loc[df_complexes['name'] == 'مجتمع هفت الماس', 'coordination'].values[0]

routing = Rosrm.route([origin, destination], alternatives='true')

In [10]:
df_routing = pd.DataFrame(routing['routes'])
df_routing

,start_point_latitude,start_point_longitude,end_point_latitude,end_point_longitude,route_number,geometry,route_coordinates,distance,duration
0,32.25749505,51.42830138753927,36.21997795,50.21559045426398,1,ab{cEu}zxHgnWsqCy|i@dya@ivMkvFihIlbGehDawJijSy...,"[(32.25649, 51.42507), (32.38181, 51.44853), (...",551057.7,22208.4
1,32.25749505,51.42830138753927,36.21997795,50.21559045426398,2,ab{cEu}zxHgnWsqCy|i@dya@ivMkvFyrIf`Gu}C{tJijSy...,"[(32.25649, 51.42507), (32.38181, 51.44853), (...",601002.0,22986.0


In [11]:
# Main Route
print(f"Distance: {df_routing['distance'][0]/1000} km\nDuration: {df_routing['duration'][0]/3600} hours")
Rosrm.get_map(df_routing['route_coordinates'][0])

Distance: 551.0577 km
Duration: 6.1690000000000005 hours


In [12]:
# Alternative Route
print(f"Distance: {df_routing['distance'][1]/1000} km\nDuration: {df_routing['duration'][1]/3600} hours")
Rosrm.get_map(df_routing['route_coordinates'][1])

Distance: 601.002 km
Duration: 6.385 hours
